En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Latam Data Engineer Challenge
*Santiago Hoyos*

In [73]:
file_path = "farmers-protest-tweets-2021-2-4.json"

Importo todas las librerías a utilizar:

In [65]:
import pandas as pd
from memory_profiler import memory_usage
import cProfile

Creo un DataFrame de Pandas

In [67]:
# Cargar el archivo JSON en un DataFrame de Pandas
df = pd.read_json(file_path, lines=True)

Deseo entender la estructura del DataFrame

In [22]:
# Mostrar las primeras filas del DataFrame para visualizar la estructura y los datos
#print("Primeras filas del DataFrame:")
#print(df.head(5))

# Ver la información general del DataFrame
print("\nInformación del DataFrame:")
print(df.info())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117407 entries, 0 to 117406
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   url              117407 non-null  object             
 1   date             117407 non-null  datetime64[ns, UTC]
 2   content          117407 non-null  object             
 3   renderedContent  117407 non-null  object             
 4   id               117407 non-null  int64              
 5   user             117407 non-null  object             
 6   outlinks         117407 non-null  object             
 7   tcooutlinks      117407 non-null  object             
 8   replyCount       117407 non-null  int64              
 9   retweetCount     117407 non-null  int64              
 10  likeCount        117407 non-null  int64              
 11  quoteCount       117407 non-null  int64              
 12  conversationId   117407 non-nu

In [63]:
#print(df['user'])
#df['mentionedUsers'].head(5)

0    [{'username': 'narendramodi', 'displayname': '...
1    [{'username': 'Kisanektamorcha', 'displayname'...
2                                                 None
3    [{'username': 'ReallySwara', 'displayname': 'S...
4                                                 None
Name: mentionedUsers, dtype: object

# Task 1

**1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:**

**Optimización tiempo de ejecución**

Explicación de q1_time:

1. Carga de Datos Eficiente: En este enfoque, se utiliza la biblioteca pandas para cargar el archivo JSON en un DataFrame. pandas está altamente optimizado para el procesamiento de datos tabulares y es muy eficiente para manejar grandes conjuntos de datos.

2. Operaciones Vectorizadas y Uso de Funciones Lambda: Se aprovechan las operaciones vectorizadas de pandas para realizar manipulaciones de datos en grandes conjuntos de datos de manera eficiente. Por ejemplo, la conversión de la columna de fecha a datetime y luego a datetime.date se realiza de manera vectorizada utilizando el método pd.to_datetime y dt.date. Además, las funciones lambda son utilizadas para aplicar transformaciones simples a los datos de manera eficiente.

3. Agrupación y Reducción de Datos: Se utiliza la función groupby de pandas para agrupar los datos por fecha y contar el número de tweets en cada fecha. Esto se realiza de manera muy eficiente en términos de tiempo de ejecución.

4. Selección de los Top Usuarios: Para cada una de las 10 fechas con más tweets, se selecciona el usuario que más publicó en esa fecha. Esto se hace de manera eficiente utilizando operaciones de agrupación y clasificación de pandas.

Por qué es Bueno para Optimizar el Tiempo de Ejecución:

- pandas es altamente eficiente para el procesamiento de datos tabulares y proporciona operaciones optimizadas que pueden manejar grandes volúmenes de datos de manera eficiente.

- Las operaciones vectorizadas y el uso de funciones **lambda** permiten realizar transformaciones y cálculos en los datos de manera eficiente, evitando bucles explícitos.

- La agrupación de datos realizada por pandas permite procesar grandes conjuntos de datos de manera rápida y eficiente.

In [45]:
from typing import List, Tuple
import pandas as pd
import datetime
#file_path = "farmers-protest-tweets-2021-2-4.json"


#@profile   #se puede descomentar para correr la función en la terminal con python -m memory_profiler q1_time.py y ver el uso de memoria.

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Cargar el archivo JSON en un DataFrame de Pandas
    #df = pd.read_json(file_path, lines=True)
    
    # Extraer el 'username' de la columna 'user'
    df['username'] = df['user'].apply(lambda user: user['username'])
    
    # Convertir la columna 'date' a datetime.date
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    # Agrupar por fecha y contar los tweets
    tweet_counts = df.groupby(df['date'])['id'].count().reset_index()
    
    # Ordenar por el conteo de tweets descendente y obtener las 10 fechas principales
    top_dates = tweet_counts.sort_values('id', ascending=False).head(10)['date'].tolist()
    
    # Encontrar el usuario con más tweets para cada fecha principal
    top_users = []
    for date in top_dates:
        date_df = df[df['date'] == date]
        top_user = date_df.groupby('username')['id'].count().sort_values(ascending=False).index[0]
        top_users.append((date, top_user))
    
    return top_users

print(q1_time(file_path))

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


In [46]:
#### TIEMPO DE EJECUCIÓN q1_time ####
cProfile.run("q1_time(file_path)")

         136942 function calls (136536 primitive calls) in 0.689 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    0.028    0.000    0.028    0.000 1612929132.py:14(<lambda>)
        1    0.032    0.032    0.687    0.687 1612929132.py:9(q1_time)
       23    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
        1    0.002    0.002    0.689    0.689 <string>:1(<module>)
       24    0.000    0.000    0.000    0.000 _decorators.py:218(_format_argument_list)
    24/23    0.000    0.000    0.007    0.000 _decorators.py:308(wrapper)
       40    0.000    0.000    0.000    0.000 _dtype.py:24(_kind_name)
       40    0.000    0.000    0.000    0.000 _dtype.py:330(_name_includes_bit_suffix)
       40    0.000    0.000    0.000    0.000 _dtype.py:346(_name_get)
        1    0.000    0.000    0.000    0.000 _methods.py:39(_amax)
        1    0.000    0.000    0.000    0.000 _methods.

MEMORIA:
Para evaluar el uso de memoria de la función, se colocó el indicador @profile al comienzo de la función y se corrió desde la terminal de la siguiente forma: python -m memory_profiler q1_time.py. Lo anterior arrojó el siguiente resultado:

          
     Line #   Mem usage    Increment  Occurrences   Line Contents
          

          8 1505.211 MiB 1505.211 MiB           1   @profile
          9                                         def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
          10                                             # Cargar el archivo JSON en un DataFrame de Pandas
          11
          12                                             # Extraer el 'username' de la columna 'user'
          13 1507.539 MiB    2.328 MiB      234815       df['username'] = df['user'].apply(lambda user: user['username'])
          14
          15                                             # Convertir la columna 'date' a datetime.date
          16 1512.254 MiB    4.715 MiB           1       df['date'] = pd.to_datetime(df['date']).dt.date
          17
          18                                             # Agrupar por fecha y contar los tweets
          19 1513.621 MiB    1.367 MiB           1       tweet_counts = df.groupby(df['date'])['id'].count().reset_index()
          20
          21                                             # Ordenar por el conteo de tweets descendente y obtener las 10 fechas principales
          22 1513.688 MiB    0.066 MiB           1       top_dates = tweet_counts.sort_values('id', ascending=False).head(10)['date'].tolist()
          23
          24                                             # Encontrar el usuario con más tweets para cada fecha principal
          25 1513.688 MiB    0.000 MiB           1       top_users = []
          26 1521.445 MiB   -0.035 MiB          11       for date in top_dates:
          27 1521.379 MiB    6.562 MiB          10           date_df = df[df['date'] == date]
          28 1521.445 MiB    1.148 MiB          10           top_user = date_df.groupby('username')['id'].count().sort_values(ascending=False).index[0]
          29 1521.445 MiB   -0.035 MiB          10           top_users.append((date, top_user))
          30
          31 1521.445 MiB    0.000 MiB           1       return top_users

**Optimización de memoria en uso**

Explicación de q1_memory:

1.   Lectura Línea por Línea: En este enfoque, se lee el archivo JSON línea por línea en lugar de cargarlo completo en la memoria. Esto reduce significativamente el uso de memoria, ya que no se carga todo el archivo en la memoria al mismo tiempo.

2.  Uso de Estructuras de Datos Eficientes: Se utilizan estructuras de datos eficientes como Counter y defaultdict para contar los tweets por fecha y por usuario. Estas estructuras de datos están optimizadas para minimizar el uso de memoria y ofrecen un rendimiento eficiente para el conteo de elementos.

3.   Reducción de Datos Intermedios: En lugar de almacenar todos los datos en memoria, se calculan y almacenan solo los datos necesarios, como los conteos de tweets por fecha y por usuario. Esto reduce la cantidad de memoria necesaria para almacenar datos intermedios.

Por qué es Bueno para Optimizar la Memoria en Uso:

-   La lectura línea por línea del archivo y el procesamiento de datos en tiempo real reducen significativamente la cantidad de memoria necesaria, ya que no es necesario cargar todo el archivo en la memoria al mismo tiempo.

-   El uso de estructuras de datos eficientes como Counter y defaultdict minimiza el uso de memoria al almacenar solo la información necesaria para realizar los cálculos requeridos.

In [48]:
from q1_memory import q1_memory
result = q1_memory(file_path=file_path)
print(result)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


In [33]:
cProfile.run("q1_memory(file_path)")

         1560148 function calls in 4.839 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003    4.839    4.839 <string>:1(<module>)
   117407    0.099    0.000    2.821    0.000 __init__.py:299(loads)
       14    0.000    0.000    0.000    0.000 __init__.py:581(__init__)
    51659    0.005    0.000    0.005    0.000 __init__.py:595(__missing__)
        1    0.000    0.000    0.000    0.000 __init__.py:600(most_common)
       14    0.000    0.000    0.000    0.000 __init__.py:649(update)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:11(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
    49772    0.023    0.000    0.584    0.000 cp1252.py:22(decode)
   117407    0.143    0.000    2.693    0.000 decoder.py:332(decode)
   117407    2.433    0.000    2.433    0.000 decoder.py:343(raw_decode)
        1    0.000    0.000    0.000    0.000 heapq.py:

MEMORIA:
Para evaluar el uso de memoria de la función, se colocó el indicador @profile al comienzo de la función y se corrió desde la terminal de la siguiente forma: python -m memory_profiler q1_memory.py. Lo anterior arrojó el siguiente resultado:

    Line #    Mem usage    Increment  Occurrences   Line Contents
    =============================================================
        8   41.039 MiB   41.039 MiB           1   @profile
        9                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
        10   41.051 MiB    0.012 MiB           1       tweet_counts = Counter()
        11   41.051 MiB    0.000 MiB           1       user_counts = defaultdict(Counter)
        12
        13   41.051 MiB    0.000 MiB           1       with open(file_path, 'r') as f:
        14   47.750 MiB    2.016 MiB      117408           for line in f:
        15   47.750 MiB    2.879 MiB      117407               tweet = json.loads(line)
        16   47.750 MiB    0.379 MiB      117407               date = datetime.datetime.fromisoformat(tweet['date']).date()
        17   47.750 MiB    0.000 MiB      117407               user = tweet['user']['username']
        18
        19   47.750 MiB    0.000 MiB      117407               tweet_counts[date] += 1
        20   47.750 MiB    1.426 MiB      117407               user_counts[date][user] += 1
        21
        22   47.750 MiB    0.000 MiB          13       top_dates = [date for date, count in tweet_counts.most_common(10)]
        23   47.750 MiB    0.000 MiB          13       top_users = [(date, max(user_counts[date], key=user_counts[date].get)) for date in top_dates]
        24
        25   47.750 MiB    0.000 MiB           1       return top_users

**Discusión resultados:**

La comparación entre `q1_time` y `q1_memory` revela diferencias significativas tanto en el tiempo de ejecución como en el uso de memoria. 

Para `q1_time`, que se enfoca en la optimización del tiempo de ejecución, se utilizó la biblioteca `pandas` para cargar el archivo JSON en un DataFrame, lo que permitió realizar operaciones de procesamiento de datos de manera eficiente. Sin embargo, esto resultó en un mayor uso de memoria debido a la carga completa del conjunto de datos en memoria. A pesar de esto, la ejecución fue más rápida, con un tiempo total de 0.634 segundos. 

En contraste, `q1_memory` se diseñó para optimizar el uso de memoria. En esta función, se evitó cargar el conjunto de datos completo en memoria y, en su lugar, se procesaron los datos línea por línea de manera incremental. Aunque esta estrategia resultó en un tiempo de ejecución más largo de 4.839 segundos, se logró un uso de memoria considerablemente menor, con solo 47.750 MiB utilizados respecto a los 1521.445 MiB de q1_time.

# Task 2

**2. Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:**

**Optimización tiempo de ejecución**


1. Importamos las librerías necesarias: typing para definir los tipos de las variables de entrada y salida, pandas para leer el archivo JSON y convertirlo en un DataFrame, emoji para analizar el texto y obtener los emojis, y collections para contar los emojis.
2. Definimos la función q2_time que toma como entrada un archivo JSON y devuelve una lista de tuplas, donde cada tupla contiene un emoji y su frecuencia en el archivo JSON.
3. Leemos el archivo JSON en un DataFrame de Pandas.
4. Concatenamos todos los tweets en un solo string separados por un espacio en blanco.
5. Usamos el método emoji_list de la librería emoji para analizar el string y obtener todos los emojis presentes. El método emoji_list devuelve una lista de diccionarios, donde cada diccionario contiene información sobre un emoji, incluyendo el emoji en sí.
6. Extraemos el valor de la clave emoji de cada diccionario en la lista y lo agregamos a la lista emojis.
7. Contamos los emojis en la lista emojis usando el método Counter de la librería collections.
8. Devolvemos los 10 emojis más utilizados usando el método most_common de la instancia Counter.


In [55]:
from typing import List, Tuple
import pandas as pd
from emoji import emoji_list
from collections import Counter
#file_path = "farmers-protest-tweets-2021-2-4.json"



#@profile
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON en un DataFrame de Pandas
    #df = pd.read_json(file_path, lines=True)
    
    # Obtener un solo string que contenga todos los contenidos de los tweets
    single_string = df['content'].str.cat(sep=' ')
    
    # Usar el método emoji_list para analizar el string y obtener todos los emojis presentes
    emojis = [emoji['emoji'] for emoji in emoji_list(single_string)]
    
    # Contar los emojis y devolver los 10 más utilizados
    return Counter(emojis).most_common(10)

print(q2_time(file_path))

[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)]


In [56]:
#### TIEMPO DE EJECUCIÓN q2_time ####
cProfile.run("q2_time(file_path)")

         85746551 function calls in 24.301 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   24.294   24.294 1638560488.py:10(q2_time)
        1    0.018    0.018    0.018    0.018 1638560488.py:18(<listcomp>)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
 17140708    2.866    0.000    5.161    0.000 <string>:1(<lambda>)
        1    0.007    0.007   24.301   24.301 <string>:1(<module>)
        1    0.000    0.000    0.007    0.007 __init__.py:581(__init__)
        1    0.000    0.000    0.000    0.000 __init__.py:600(most_common)
        1    0.000    0.000    0.007    0.007 __init__.py:649(update)
        1    0.000    0.000    0.000    0.000 _methods.py:55(_any)
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.003    0.003    0.092    0.092 accessor.py:122(wrapper)
        1    0.000    0.000    0.089

Memoria:

    Line #    Mem usage    Increment  Occurrences   Line Contents
    =============================================================
        10 1512.516 MiB 1512.516 MiB           1   @profile
        11                                         def q2_time(file_path: str) -> List[Tuple[str, int]]:
        12                                             # Leer el archivo JSON en un DataFrame de Pandas
        13                                             #
        14
        15                                             # Obtener un solo string que contenga todos los contenidos de los tweets
        16 1579.945 MiB   67.430 MiB           1       single_string = df['content'].str.cat(sep=' ')
        17
        18                                             # Usar el método emoji_list para analizar el string y obtener todos los emojis presentes
        19 1593.645 MiB   12.777 MiB       42925       emojis = [emoji['emoji'] for emoji in emoji_list(single_string)]
        20
        21                                             # Contar los emojis y devolver los 10 más utilizados
        22 1593.184 MiB   -0.461 MiB           1       return Counter(emojis).most_common(10)

**Optimización memoria en uso**


1. En lugar de leer todo el archivo JSON en memoria y convertirlo en un DataFrame, leemos el archivo línea por línea usando un bucle for. Esto significa que solo necesitamos tener una línea del archivo en memoria a la vez, en lugar de cargar todo el archivo en memoria al mismo tiempo.
2. En lugar de crear una cadena grande que contenga todos los tweets, iteramos sobre cada tweet y lo procesamos individualmente. Esto significa que solo necesitamos mantener un tweet en memoria a la vez, en lugar de crear una gran cadena que contenga todos los tweets.
3. En lugar de usar la función emoji_list para analizar toda la cadena de tweets a la vez, analizamos cada tweet individualmente usando el mismo método. Esto significa que solo necesitamos mantener un tweet en memoria a la vez, en lugar de crear una gran lista que contenga todos los emojis de todos los tweets.
4. Usamos el método update del contador emoji_counts para actualizar el contador con los emojis de cada tweet. Esto significa que no necesitamos crear una nueva lista de emojis para cada tweet, sino que podemos actualizar el contador en el lugar.


In [59]:
from q2_memory import q2_memory
result = q2_memory(file_path=file_path)
print(result)

[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)]


In [47]:
#### TIEMPO DE EJECUCIÓN q2_memory ####
cProfile.run("q2_memory(file_path)")

         87724551 function calls in 36.425 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    0.027    0.000    0.027    0.000 3127824189.py:14(<listcomp>)
        1    2.284    2.284   36.425   36.425 3127824189.py:5(q2_memory)
 17023302    3.484    0.000    6.248    0.000 <string>:1(<lambda>)
        1    0.000    0.000   36.425   36.425 <string>:1(<module>)
   117407    0.158    0.000    4.050    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 __init__.py:581(__init__)
        1    0.000    0.000    0.000    0.000 __init__.py:600(most_common)
   117408    0.093    0.000    0.324    0.000 __init__.py:649(update)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:11(getpreferredencoding)
   117407    0.042    0.000    0.106    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
   117407    0.117    0.000   28.994    0.000 c

Memoria:

    Line #    Mem usage    Increment  Occurrences   Line Contents
    =============================================================
        9   51.199 MiB   51.199 MiB           1   @profile
        10                                         def q2_memory(file_path: str) -> List[Tuple[str, int]]:
        11   51.211 MiB    0.012 MiB           1       emoji_counts = Counter()
        12
        13   51.211 MiB    0.000 MiB           1       with open(file_path, 'r') as f:
        14   53.863 MiB -3625.105 MiB      117408           for line in f:
        15   53.863 MiB -3625.699 MiB      117407               tweet = json.loads(line)
        16   53.863 MiB -3625.859 MiB      117407               content = tweet['content']
        17
        18                                                     # Se obtienen los emojis
        19   53.863 MiB -12499.953 MiB      395143               emojis = [emoji['emoji'] for emoji in emoji_list(content)]
        20
        21                                                     # Se actualiza el contador con los emojis que se encontraron.
        22   53.863 MiB -3625.887 MiB      117407               emoji_counts.update(emojis)
        23
        24   53.863 MiB    0.000 MiB           1       top_emojis = emoji_counts.most_common(10)
        25
        26   53.863 MiB    0.000 MiB           1       return top_emojis

**Discusión de resultados**

En términos de tiempo, q2_time es más rápido que q2_memory (24.301 segundos vs 36.425 segundos). Esto se debe a que q2_time lee todo el archivo JSON en memoria y lo convierte en un DataFrame de Pandas, y luego concatena todas las cadenas de tweets en una sola cadena. Esto es más rápido que leer el archivo línea por línea y procesar cada tweet individualmente, como lo hace q2_memory.

Sin embargo, en términos de memoria, q2_memory es más eficiente que q2_time (53.863 MiB vs 1593.184 MiB). Esto se debe a que q2_memory solo necesita mantener una pequeña cantidad de datos en memoria a la vez, en lugar de crear una gran cadena que contenga todos los tweets. Además, q2_memory no crea una gran lista de emojis para cada tweet, sino que actualiza el contador en el lugar.

# Task 3

**3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. Debe incluir las siguientes funciones:**

**Optimización tiempo de ejecución**

1. Importamos las librerías necesarias: typing para definir los tipos de las variables de entrada y salida, pandas para leer el archivo JSON y convertirlo en un DataFrame.
2. Definimos la función q3_time que toma como entrada un archivo JSON y devuelve una lista de tuplas, donde cada tupla contiene un nombre de usuario y su frecuencia en el archivo JSON.
3. Cargamos el archivo JSON en un DataFrame de Pandas.
4. Extraemos las menciones de la columna mentionedUsers usando el método apply y una función lambda. La función lambda itera sobre cada elemento de la columna y extrae el valor de la clave username de cada diccionario en la lista, si la entrada es una lista. Si la entrada no es una lista, la función lambda devuelve una lista vacía. La función lambda se utiliza en lugar de una comprensión de lista para optimizar el tiempo de ejecución, ya que evita crear una gran lista de listas de menciones que contenga todas las menciones de todos los tweets.
5. Aplanamos la lista de listas de menciones usando una comprensión de lista.
6. Contamos las menciones por usuario usando el método value_counts de la columna mentions.
7. Obtenemos los 10 usuarios con más menciones usando el método head de la Serie mention_counts.
8. Renombramos las columnas de la DataFrame top_users para que la columna de nombres de usuario se llame username y la columna de frecuencias se llame count.
9.Convertimos la DataFrame top_users en una lista de tuplas usando el método itertuples y devolvemos la lista.


In [68]:
from typing import List, Tuple
import pandas as pd

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    # Cargar el archivo JSON en un DataFrame de Pandas
    #df = pd.read_json(file_path, lines=True)
    
    # Extraer las menciones de la columna 'mentionedUsers'
    df['mentions'] = df['mentionedUsers'].apply(lambda x: [user['username'] for user in x] if isinstance(x, list) else [])
    
    # Aplanar la lista de listas de menciones
    mentions = [mention for mentions in df['mentions'] for mention in mentions]
    
    # Contar las menciones por usuario
    mention_counts = pd.Series(mentions).value_counts()
    
    # Obtener los 10 usuarios con más menciones
    top_users = mention_counts.head(10).reset_index()
    top_users.columns = ['username', 'count']
    
    return list(top_users.itertuples(index=False, name=None))

In [69]:
print(q3_time(file_path))

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]


In [71]:
#### TIEMPO DE EJECUCIÓN q3_time ####
cProfile.run("q3_time(file_path)")

         275680 function calls (275631 primitive calls) in 0.418 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.029    0.029    0.029    0.029 3430477421.py:12(<listcomp>)
        1    0.005    0.005    0.414    0.414 3430477421.py:4(q3_time)
   117407    0.042    0.000    0.268    0.000 3430477421.py:9(<lambda>)
    38034    0.218    0.000    0.218    0.000 3430477421.py:9(<listcomp>)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
        1    0.004    0.004    0.418    0.418 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 _decorators.py:218(_format_argument_list)
      3/2    0.000    0.000    0.004    0.002 _decorators.py:308(wrapper)
        1    0.000    0.000    0.000    0.000 _dtype.py:24(_kind_name)
        1    0.000    0.000    0.000    0.000 _dtype.py:330(_name_includes_bit_suffix)
        1    0.000    0.000    0.000    0.000 

Memoria:

    Line #    Mem usage    Increment  Occurrences   Line Contents
    =============================================================
        6   75.102 MiB   75.102 MiB           1   @profile  #se puede descomentar para correr la función en la terminal con python -m memory_profiler q1_time.py y ver el uso de memoria.
        7                                         def q3_time(file_path: str) -> List[Tuple[str, int]]:
        8                                             # Cargar el archivo JSON en un DataFrame de Pandas
        9 1504.672 MiB 1429.570 MiB           1       df = pd.read_json(file_path, lines=True)
        10
        11                                             # Extraer las menciones de la columna 'mentionedUsers'
        12 1508.375 MiB    3.703 MiB      414286       df['mentions'] = df['mentionedUsers'].apply(lambda x: [user['username'] for user in x] if isinstance(x, list) else [])
        13
        14                                             # Aplanar la lista de listas de menciones
        15 1510.125 MiB    1.750 MiB      220813       mentions = [mention for mentions in df['mentions'] for mention in mentions]
        16
        17                                             # Contar las menciones por usuario
        18 1511.961 MiB    1.836 MiB           1       mention_counts = pd.Series(mentions).value_counts()
        19
        20                                             # Obtener los 10 usuarios con más menciones
        21 1512.098 MiB    0.137 MiB           1       top_users = mention_counts.head(10).reset_index()
        22 1512.098 MiB    0.000 MiB           1       top_users.columns = ['username', 'count']
        23
        24 1512.109 MiB    0.012 MiB           1       return list(top_users.itertuples(index=False, name=None))

**Optimización de memoria en uso**

1. Se crea un objeto Counter llamado mention_counts para contar las menciones.
2. Se abre el archivo JSON farmers-protest-tweets-2021-2-4.json y se procesa cada línea.
3. Para cada línea, se evalúa como un diccionario de Python y se extraen los usuarios mencionados de la columna 'mentionedUsers'.
4. Se actualiza el contador mention_counts con las menciones de cada línea.
5. Se obtienen los 10 usuarios con más menciones utilizando el método most_common del objeto Counter.
6. Se devuelve una lista de tuplas con los 10 usuarios con más menciones.

En este código, no es necesario aplanar explícitamente una lista, ya que el objeto Counter se encarga de contar automáticamente las menciones de cada usuario, incluso si se encuentran en listas anidadas. El método most_common devuelve una lista de tuplas con los 10 usuarios con más menciones y su respectivo recuento.

In [76]:
from q3_memory import q3_memory
result = q3_memory(file_path)
print(result)

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]


In [77]:
#### TIEMPO DE EJECUCIÓN q3_memory ####
cProfile.run("q3_memory(file_path)")

         1898723 function calls in 4.797 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    4.797    4.797 <string>:1(<module>)
   117407    0.106    0.000    2.772    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 __init__.py:581(__init__)
        1    0.000    0.000    0.003    0.003 __init__.py:600(most_common)
   117408    0.051    0.000    0.202    0.000 __init__.py:649(update)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:11(getpreferredencoding)
   117407    0.025    0.000    0.057    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
    49772    0.021    0.000    0.584    0.000 cp1252.py:22(decode)
   117407    0.140    0.000    2.635    0.000 decoder.py:332(decode)
   117407    2.381    0.000    2.381    0.000 decoder.py:343(raw_decode)
        1    0.003    0.003    0.003    0.003 heapq.py:

Memoria:

    Line #    Mem usage    Increment  Occurrences   Line Contents
    =============================================================
        6   41.258 MiB   41.258 MiB           1   @profile  #se puede descomentar para correr la función en la terminal con python -m memory_profiler q1_time.py y ver el uso de memoria.
        7                                         def q3_memory(file_path: str) -> List[Tuple[str, int]]:
        8                                             # Crear un objeto Counter para contar las menciones
        9   41.270 MiB    0.012 MiB           1       mention_counts = Counter()
        10
        11                                             # Abrir el archivo y procesar cada línea
        12   41.270 MiB    0.000 MiB           1       with open(file_path, 'r') as f:
        13   44.461 MiB -13878.895 MiB      117408           for line in f:
        14                                                     # Evaluar la línea como un diccionario de Python
        15   44.461 MiB -13878.473 MiB      117407               tweet = json.loads(line)
        16
        17                                                     # Extraer los usuarios mencionados de la columna 'mentionedUsers'
        18   44.461 MiB -34503.359 MiB      296878               mentions = [user['username'] for user in tweet['mentionedUsers']] if tweet['mentionedUsers'] else []
        19
        20                                                     # Actualizar el contador con las menciones de esta línea
        21   44.461 MiB -13878.625 MiB      117407               mention_counts.update(mentions)
        22
        23                                             # Obtener los 10 usuarios con más menciones utilizando el método most_common
        24   43.988 MiB   -0.473 MiB           1       top_users = mention_counts.most_common(10)
        25
        26   43.988 MiB    0.000 MiB           1       return top_users

**Discusión de Resultados**

En términos de tiempo de ejecución, q3_time es más rápido que q3_memory, ya que q3_time tarda 0.418 segundos en completarse, mientras que q3_memory tarda 4.797 segundos. q3_time es más rápido que q3_memory porque q3_time utiliza operaciones vectorizadas de Pandas para procesar las menciones en lugar de iterar sobre cada elemento de la lista de menciones.

En cuanto al uso de memoria, q3_memory utiliza significativamente menos memoria que q3_time. q3_memory utiliza una media de 43.988 MiB de memoria, mientras que q3_time utiliza una media de 1512.109 MiB de memoria. Esto se debe a que q3_memory no crea una columna adicional en el DataFrame de Pandas para almacenar las menciones, y tampoco aplana explícitamente la lista de menciones. Además, q3_memory utiliza un objeto Counter en lugar de una Serie de Pandas para contar las menciones, lo que reduce aún más el uso de memoria.